In [1]:
import speechbrain
import torchaudio
import numpy as np
import tensorflow

import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Flatten, Dropout, Activation
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.layers import BatchNormalization, ELU, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split

In [3]:
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(1)

### Entendiendo SpeechBrain

In [4]:
import torch
import speechbrain as sb

class SimpleBrain(sb.Brain):
    def compute_forward(self, batch, stage):
        return self.modules.model(batch["input"])

    
    def compute_objectives(self, predictions, batch, stage):
        return torch.nn.functional.l1_loss(predictions, batch["target"])

model = torch.nn.Linear(in_features=10, out_features=10)
brain = SimpleBrain({"model": model}, opt_class=lambda x: torch.optim.SGD(x, 0.1))
data = [{"input": torch.rand(10, 10), "target": torch.rand(10, 10)}]
brain.fit(range(10), data)

100%|██████████| 1/1 [00:00<00:00, 277.58it/s, train_loss=0.409]


### Como defino una clase de speechbrain

```
def __init__(
    self,
    modules=None,
    opt_class=None,
    hparams=None,
    run_opts=None,
    checkpointer=None,
):
```

In [5]:
import torch
from speechbrain.nnet.complex_networks.c_CNN import CConv1d, CConv2d
from speechbrain.nnet.complex_networks.c_linear import CLinear

In [ ]:
# cnn_1d_new = CConv1d( input_shape=input_X.shape, out_channels=12, kernel_size=1, padding = 'valid')
# lin = CLinear(10, input_shape=out_tensor.shape, init_criterion='glorot', weight_init='complex')

### Ejemplo de un clasificador simple
```python
from speechbrain.lobes.features import MFCC, Fbank
from speechbrain.nnet.losses import nll_loss


class SimpleBrain(speechbrain.Brain):
    def compute_forward(self, batch, stage):
        example_batch = batch
        x = self.modules.features(batch.signal.data)
        x = self.modules.encoder(x)
        x = self.modules.pooling(x, batch.signal.lengths)
        x = self.modules.to_output(x)
        return self.modules.softmax(x)
    
    def compute_objectives(self, logits, batch, stage):
        return nll_loss(logits, batch.spk_encoded.data)

modules = {"features": Fbank(left_frames=1, right_frames=1),
          "encoder": torch.nn.Sequential(torch.nn.Linear(40, 256),
                                       torch.nn.ReLU()),
          "pooling": speechbrain.nnet.pooling.StatisticsPooling(),
          "to_output": torch.nn.Linear(512, len(spk_id_encoder)),
          "softmax": speechbrain.nnet.activations.Softmax(apply_log=True)}
brain = SimpleBrain(modules, opt_class=lambda x: torch.optim.SGD(x, 1))
brain.fit(range(1), train_set=sorted_data, 
          train_loader_kwargs={"batch_size": 8, "drop_last":True})
```

### Tratando de crear un clasificador complejo simple!

In [6]:
from speechbrain.nnet.complex_networks.c_ops import get_real, get_imag